In [1]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
import time
from sklearn import svm
from sklearn.metrics import recall_score, f1_score, precision_score, accuracy_score
import nltk
from nltk.corpus import stopwords
import xgboost as xgb

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Alienware\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alienware\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Alienware\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Alienware\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
train_data = []  # 16659
test_data = []  # 810

In [4]:
'''读取文件'''
print('Reading files...')
print('\n')
train_f = open("train.txt", encoding='utf-8')
for line in train_f:
    train_data.append(line)
train_f.close()

test_f = open("test.txt", encoding='utf-8')
for line in test_f:
    test_data.append(line)
test_f.close()
print(train_data[0])
print(test_data[0])
print('\n')
print('Files get!...')

Reading files...


" 5 . Science includes such diverse fields as astronomy , biology , computer sciences , geology , logic , physics , chemistry , and mathematics ( [ link ] ) ."	"0"

" 2 . It becomes clear from this definition that the application of the scientific method plays a major role in science ."	"0"



Files get!...


In [5]:
'''去掉开头序号-> [[句子(单词间以,为分割)], 0/1]'''
print('Formatting...')
print('\n')
train_set = []
for unit in train_data:
    temp = unit.split(' ')
    if temp[1].isdigit():
        # 判断第一个字符是否为数字，如果是则去掉，如果不是则取句子
        temp = temp[3:-1]
        # 当前temp中的最后一部分为"\t.\n","最后一个引号" 为了保证格式规范，先去掉，最后在取corpus的时候加上句号即可
        # training_set.append(([temp, unit[-3:-2]]))
        train_set.append([' '.join(temp), unit[-3:-2]])
        # unit[-3:-2]为当前句子的label
    else:
        # temp[1]为引号，应去掉
        temp = temp[1:-1]
        # training_set.append(([temp, unit[-3:-2]]))
        train_set.append([' '.join(temp), unit[-3:-2]])

test_set = []
for unit in test_data:
    temp = unit.split(' ')
    if temp[1].isdigit():
        temp = temp[3:-1]
        # training_set.append(([temp, unit[-3:-2]]))
        test_set.append([' '.join(temp), unit[-3:-2]])
    else:
        temp = temp[1:-1]
        # training_set.append(([temp, unit[-3:-2]]))
        test_set.append([' '.join(temp), unit[-3:-2]])

print(train_set[0])
print(test_set[0])
print('\n')
print('All data sets are formatted!')

Formatting...


['Science includes such diverse fields as astronomy , biology , computer sciences , geology , logic , physics , chemistry , and mathematics ( [ link ] )', '0']
['It becomes clear from this definition that the application of the scientific method plays a major role in science', '0']


All data sets are formatted!


In [6]:
print('Tokenizeing...')
print('\n')

def tokenize(data):
    res = []
    for samples in data:
        # nltk.word_tokenize用于取tokens
        temp_t = nltk.word_tokenize(samples[0])
        res.append([temp_t, samples[1]])
    return res


train_set = tokenize(train_set)
test_set = tokenize(test_set)
print(train_set[0])
print(test_set[0])
print('\n')
print('Tokenization completed!')

Tokenizeing...


[['Science', 'includes', 'such', 'diverse', 'fields', 'as', 'astronomy', ',', 'biology', ',', 'computer', 'sciences', ',', 'geology', ',', 'logic', ',', 'physics', ',', 'chemistry', ',', 'and', 'mathematics', '(', '[', 'link', ']', ')'], '0']
[['It', 'becomes', 'clear', 'from', 'this', 'definition', 'that', 'the', 'application', 'of', 'the', 'scientific', 'method', 'plays', 'a', 'major', 'role', 'in', 'science'], '0']


Tokenization completed!


In [7]:
print('Removing stopwords...')
print('\n')
stop_words = set(stopwords.words("english"))
# stop_words为英文所有stop words的集合


def Remove_stopwords(data):
    res = []
    temp = []
    for sets in data:
        for w in sets[0]:
            # 如果对于句子中任意一个单词不属于stop_words则将其加入新的dataset
            if w not in stop_words:
                temp.append(w)
        res.append([temp, sets[1]])
        temp = []

    return res


train_set = Remove_stopwords(train_set)
test_set = Remove_stopwords(test_set)
print(train_set[0])
print(test_set[0])
print('\n')
print('Stopwords removed!')

Removing stopwords...


[['Science', 'includes', 'diverse', 'fields', 'astronomy', ',', 'biology', ',', 'computer', 'sciences', ',', 'geology', ',', 'logic', ',', 'physics', ',', 'chemistry', ',', 'mathematics', '(', '[', 'link', ']', ')'], '0']
[['It', 'becomes', 'clear', 'definition', 'application', 'scientific', 'method', 'plays', 'major', 'role', 'science'], '0']


Stopwords removed!


In [8]:
print('Stemming...')
print('\n')
ps = PorterStemmer()


def Stemming(data):
    res = []
    stemmed_words_temp = []
    for units in data:
        for w in units[0]:
            #ps.stem(w):对当前单词w取stem word
            stemmed_words_temp.append(ps.stem(w))

        res.append([stemmed_words_temp, units[1]])
        stemmed_words_temp = []
    return res


train_set = Stemming(train_set)
test_set = Stemming(test_set)
print(train_set[0])
print(test_set[0])
print('\n')
print('Stemming completed!')

Stemming...


[['scienc', 'includ', 'divers', 'field', 'astronomi', ',', 'biolog', ',', 'comput', 'scienc', ',', 'geolog', ',', 'logic', ',', 'physic', ',', 'chemistri', ',', 'mathemat', '(', '[', 'link', ']', ')'], '0']
[['It', 'becom', 'clear', 'definit', 'applic', 'scientif', 'method', 'play', 'major', 'role', 'scienc'], '0']


Stemming completed!


In [9]:
print('Lemmatization...')
print('\n')
lem = WordNetLemmatizer()
stem = PorterStemmer()


def Lemmatization(data):
    lem_words_temp = []
    res = []
    for atoms in data:
        for w in atoms[0]:
            #lem.lemmatize(w, "v"): 对当前单词w取Lemmatization
            lem_words_temp.append(lem.lemmatize(w, "v"))
        res.append([lem_words_temp, atoms[1]])
        lem_words_temp = []

    return res


train_set = Lemmatization(train_set)
test_set = Lemmatization(test_set)
print(train_set[0])
print(test_set[0])
print('\n')
print('Lemmatization completed!')

Lemmatization...


[['scienc', 'includ', 'divers', 'field', 'astronomi', ',', 'biolog', ',', 'comput', 'scienc', ',', 'geolog', ',', 'logic', ',', 'physic', ',', 'chemistri', ',', 'mathemat', '(', '[', 'link', ']', ')'], '0']
[['It', 'becom', 'clear', 'definit', 'applic', 'scientif', 'method', 'play', 'major', 'role', 'scienc'], '0']


Lemmatization completed!


In [10]:
print('Corpus and labels gathering...')
print('\n')
corpus_test = []
corpus_train = []


def Gathering_corpus(data):
    res = []
    temp_res = []
    for aas in data:
        #corpus中仅仅需要第一个unit，即句子
        temp_res.append(aas[0])

    for aaas in temp_res:
        # 对于scikit-learn中的vectorizer，corpus必须为一句一句话的集合，而并非tokens。所以这里必须join然后加上句号。
        temp_ff = ' '.join(aaas)
        temp_ff = temp_ff + '.'
        res.append(temp_ff)
    return res


corpus_train = Gathering_corpus(train_set)
corpus_test = Gathering_corpus(test_set)
# train和test的corpus之和，成为corpus_sum
corpus_sum = []
corpus_sum = corpus_train
for things in corpus_test:
    corpus_sum.append(things)
print('len of corpus_sum:', len(corpus_sum))
print(corpus_sum[:5])
#取出label: Y_train and Y_test
Y_train = []
for thing in train_set:
    Y_train.append(thing[1])
Y_train = np.array(Y_train)
Y_test = []
for that in test_set:
    Y_test.append(that[1])
Y_test = np.array(Y_test)
print(Y_train[:5])
print(Y_test[:5])
print(Y_train.shape)
print(Y_test.shape)
print('\n')
print('Corpus and labels get!')

Corpus and labels gathering...


len of corpus_sum: 17469
['scienc includ divers field astronomi , biolog , comput scienc , geolog , logic , physic , chemistri , mathemat ( [ link ] ).', 'howev , field scienc relat physic world phenomena process consid natur scienc.', 'thu , museum natur scienc might contain item list.', 'In deduct reason , pattern think move opposit direct compar induct reason.', 'deduct reason form logic think use gener principl law forecast specif result.']
['0' '1' '0' '0' '1']
['0' '1' '0' '1' '1']
(16659,)
(810,)


Corpus and labels get!


In [11]:
print('tfidf processing...')
print('\n')
vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(corpus_sum)
X_train_tfidf = X_tfidf[:16659]
X_test_tfidf = X_tfidf[16659:17469]
X_train_tfidf = X_train_tfidf.toarray()
X_test_tfidf = X_test_tfidf.toarray()
X_tfidf_arr = X_tfidf.toarray()
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)
print(X_tfidf_arr.shape)
print('\n')
print('tfidf get!')

tfidf processing...


(16659, 14675)
(810, 14675)
(17469, 14675)


tfidf get!


In [12]:
from xgboost import XGBClassifier
import xgboost as xgb

In [13]:
def Format_into_array(data):
    res = []
    for items in data:
        temp = int(items)
        res.append(temp)
    res = np.array(res)
    
    return res

In [14]:
Y_test = Format_into_array(Y_test)
Y_train = Format_into_array(Y_train)
print(Y_test[:10])
print(Y_train[:10])

[0 1 0 1 1 1 0 0 1 0]
[0 1 0 0 1 0 0 1 1 1]


In [15]:
dtrain=xgb.DMatrix(X_train_tfidf,label=Y_train)
dtest=xgb.DMatrix(X_test_tfidf, label=Y_test)

In [16]:
params={'booster':'gbtree',
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'max_depth':30,
        'lambda':15,
        'subsample':0.75,
        'colsample_bytree':0.75,
        'min_child_weight':1.75,
        'eta': 0.025,
        'seed':0,
        'silent':1,
        'gamma':0.15,
        'learning_rate' : 0.02}

In [17]:
watchlist = [(dtrain,'train'), (dtest,'test')]

In [ ]:
bst=xgb.train(params,dtrain,num_boost_round=500,evals=watchlist)

[0]	train-auc:0.6691	test-auc:0.664249
[1]	train-auc:0.711755	test-auc:0.69491
[2]	train-auc:0.732849	test-auc:0.720166
[3]	train-auc:0.737611	test-auc:0.722608
[4]	train-auc:0.749213	test-auc:0.730803
[5]	train-auc:0.755788	test-auc:0.734316
[6]	train-auc:0.759711	test-auc:0.735575
[7]	train-auc:0.761815	test-auc:0.739927
[8]	train-auc:0.76284	test-auc:0.741902
[9]	train-auc:0.763928	test-auc:0.74671
[10]	train-auc:0.763744	test-auc:0.74492
[11]	train-auc:0.765004	test-auc:0.743481
[12]	train-auc:0.767304	test-auc:0.746513
[13]	train-auc:0.768289	test-auc:0.74535
[14]	train-auc:0.7679	test-auc:0.745183
[15]	train-auc:0.768074	test-auc:0.743638
[16]	train-auc:0.767778	test-auc:0.744135
[17]	train-auc:0.768472	test-auc:0.744371
[18]	train-auc:0.768792	test-auc:0.742406
[19]	train-auc:0.77018	test-auc:0.743453
[20]	train-auc:0.770447	test-auc:0.743102
[21]	train-auc:0.772356	test-auc:0.747423
[22]	train-auc:0.77218	test-auc:0.747662
[23]	train-auc:0.772539	test-auc:0.748416
[24]	train-au

[195]	train-auc:0.837797	test-auc:0.7833
[196]	train-auc:0.838005	test-auc:0.783368
[197]	train-auc:0.838212	test-auc:0.783388
[198]	train-auc:0.838429	test-auc:0.783402
[199]	train-auc:0.838555	test-auc:0.783276
[200]	train-auc:0.838761	test-auc:0.783317
[201]	train-auc:0.83891	test-auc:0.783774
[202]	train-auc:0.839141	test-auc:0.783736
[203]	train-auc:0.839306	test-auc:0.783716
[204]	train-auc:0.839487	test-auc:0.783886
[205]	train-auc:0.839647	test-auc:0.783995
[206]	train-auc:0.839792	test-auc:0.78433
[207]	train-auc:0.839999	test-auc:0.784534
[208]	train-auc:0.840125	test-auc:0.784548
[209]	train-auc:0.840409	test-auc:0.784684
[210]	train-auc:0.840518	test-auc:0.784868
[211]	train-auc:0.840635	test-auc:0.78495
[212]	train-auc:0.840784	test-auc:0.784916
[213]	train-auc:0.841102	test-auc:0.784957
[214]	train-auc:0.841323	test-auc:0.784759
[215]	train-auc:0.841499	test-auc:0.784964
[216]	train-auc:0.841678	test-auc:0.785046
[217]	train-auc:0.841825	test-auc:0.784991
[218]	train-auc:

[387]	train-auc:0.866602	test-auc:0.794677
[388]	train-auc:0.866753	test-auc:0.794616
[389]	train-auc:0.866881	test-auc:0.794841
[390]	train-auc:0.867012	test-auc:0.794923
[391]	train-auc:0.867105	test-auc:0.795053
[392]	train-auc:0.867278	test-auc:0.794991
[393]	train-auc:0.867356	test-auc:0.795025
[394]	train-auc:0.867454	test-auc:0.79495
[395]	train-auc:0.867564	test-auc:0.795087
[396]	train-auc:0.867684	test-auc:0.795046
[397]	train-auc:0.867792	test-auc:0.794998
[398]	train-auc:0.867895	test-auc:0.795039
[399]	train-auc:0.868018	test-auc:0.795203
[400]	train-auc:0.868141	test-auc:0.795244
[401]	train-auc:0.868206	test-auc:0.795319
[402]	train-auc:0.868332	test-auc:0.79525
[403]	train-auc:0.868428	test-auc:0.795175
[404]	train-auc:0.868498	test-auc:0.795169
[405]	train-auc:0.868614	test-auc:0.795148
[406]	train-auc:0.868729	test-auc:0.795148
[407]	train-auc:0.868891	test-auc:0.795128
[408]	train-auc:0.868999	test-auc:0.794991
[409]	train-auc:0.869111	test-auc:0.794943
[410]	train-a

In [ ]:
def Curve_for_hat(data, cutoff):
    res = []
    for items in data:
        if items >= cutoff:
            temp = 1
        else:
            temp = 0
        res.append(temp)
    res = np.array(res)
    
    return res

In [ ]:
Y_hat=bst.predict(dtest)

In [ ]:
Y_hat = Curve_for_hat(Y_hat,0.3)

In [ ]:
print('Scores of XGB:\n')
print('F1 score:', f1_score(Y_test, Y_hat, average='binary', pos_label=1))
print('Precision:', precision_score(Y_test, Y_hat, average='binary', pos_label=1))
print('Recall:', recall_score(Y_test, Y_hat, average='binary', pos_label=1))
print('Accuracy:', accuracy_score(Y_test, Y_hat))